In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA"

## 1. GO term analysis for each set of comparison

In [3]:
###----- Create reverse comparison for each, because GO term is one sided
if (FALSE) {
    ###----- Find all de files
    de.out.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name'
    de.files <- list.files(de.out.dir, pattern="*gn.csv", full.names=TRUE,recursive=TRUE)

    use.cols <- c("gene_name", "log2FoldChange", "padj")

    for (de.file in de.files) {
        reverse.name <- gsub(".csv","_rev.csv", de.file)
        print(reverse.name)
        de.df <- read_csv(de.file)
        de.df$log2FoldChange <- -de.df$log2FoldChange
        de.df$stat <- -de.df$stat
        write_csv(de.df, reverse.name)
    }
}

In [4]:
##########----- Parameters-----##########
genome <- "mm"
#########################################
sum.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/2_DE_GO'
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

###----- Find all de files
de.out.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name'
de.files <- list.files(de.out.dir, pattern="*.csv", full.names=TRUE,recursive=TRUE)

use.cols <- c("gene_name", "log2FoldChange", "padj")

In [5]:
###----- Select comparisons
timepoints <- c("Naive","48h","EE","DP","MP","TE")
de.use.files <- c()

# Per timepoint KO vs WT
for (i in timepoints) {
    # i timepoint KO vs WT
    de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("KO_", i, "-WT_", i, "-Salmon_gn.csv", sep="")))
    de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("WT_", i, "-KO_", i, "-Salmon_gn.csv", sep="")))
}

# Act timepoint vs naive
act.timepoints <- c("48h","EE","DP","MP","TE")
for (i in act.timepoints){
    for (j in c("Naive")){
        if (i != j) {
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("KO_", i, "-KO_", j, "-Salmon_gn.csv", sep="")))
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("WT_", i, "-WT_", j, "-Salmon_gn.csv", sep="")))
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("KO_", j, "-KO_", i, "-Salmon_gn.csv", sep="")))
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("WT_", j, "-WT_", i, "-Salmon_gn.csv", sep="")))
        }
    }
}

# D8 one versus another
d8.timepoints <- c("EE","DP","MP","TE")
for (i_idx in c(1:4)){
    for (j_idx in c(1:4)){
        if (i_idx != j_idx) {
            i <- d8.timepoints[i_idx]
            j <- d8.timepoints[j_idx]
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("KO_", i, "-KO_", j, "-Salmon_gn.csv", sep="")))
            de.use.files <- c(de.use.files, file.path(de.out.dir,  paste("WT_", i, "-WT_", j, "-Salmon_gn.csv", sep="")))
        }
    }
}

de.files.slt <- intersect(de.files, de.use.files)
print(length(de.files.slt ))
de.files.slt

[1] 56


[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_48h-KO_Naive-Salmon_gn.csv"  
 [2] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_48h-WT_48h-Salmon_gn.csv"    
 [3] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_DP-KO_EE-Salmon_gn.csv"      
 [4] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_DP-KO_MP-Salmon_gn.csv"      
 [5] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_DP-KO_Naive-Salmon_gn.csv"   
 [6] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_DP-KO_TE-Salmon_gn.csv"      
 [7] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_DP-WT_DP-Salmon_gn.csv"      
 [8] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_EE-KO_DP-Salmon_gn.csv"      
 [9] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_EE-KO_MP-Salmon_gn.csv"      
[10] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_EE-KO_Naive-Salmon_gn.csv"   
[11] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_EE-KO_TE-Salmon_gn.csv"      
[12] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_EE-WT_EE-Salmon_gn.csv"      
[13] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_MP-KO_DP-Salmon_gn.csv"      
[14] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_MP-KO_EE-Salmon_gn.csv"      
[15] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_MP-KO_Naive-Salmon_gn.csv"   
[16] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_MP-KO_TE-Salmon_gn.csv"      
[17] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_MP-WT_MP-Salmon_gn.csv"      
[18] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-KO_48h-Salmon_gn.csv"  
[19] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-KO_DP-Salmon_gn.csv"   
[20] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-KO_EE-Salmon_gn.csv"   
[21] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-KO_MP-Salmon_gn.csv"   
[22] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-KO_TE-Salmon_gn.csv"   
[23] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_Naive-WT_Naive-Salmon_gn.csv"
[24] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_TE-KO_DP-Salmon_gn.csv"      
[25] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_TE-KO_EE-Salmon_gn.csv"      
[26] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_TE-KO_MP-Salmon_gn.csv"      
[27] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_TE-KO_Naive-Salmon_gn.csv"   
[28] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/KO_TE-WT_TE-Salmon_gn.csv"      
[29] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/WT_48h-KO_48h-Salmon_gn.csv"    
[30] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/WT_48h-WT_Naive-Salmon_gn.csv"  
[31] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/WT_DP-KO_DP-Salmon_gn.csv"      
[32] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_DE_out/2_DEseq/gene_name/WT_DP-WT_EE-Salmon_gn.csv"      
[33] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA/0_salmon_

In [7]:
for (file.i in de.files.slt) {
    
    ###----- Create output names & output directory
    file.i.outpath <- gsub(de.out.dir, "", file.i)
    file.i.outpath <- gsub(".csv", "", file.i.outpath)
    file.i.outpath <- gsub("^/", "",file.i.outpath)

    file.i.simp.name <- tail(unlist(strsplit( file.i.outpath, "/")), 1)
    file.i.simp.name <- gsub("_differential", "", file.i.simp.name)
    file.i.simp.vec <- unlist(strsplit(file.i.simp.name, "_vs_"))

    # Create output directory
    wk.dir <- file.path(sum.dir, file.i.outpath)
    dir.create(wk.dir, showWarnings = FALSE, recursive=TRUE)
    setwd(wk.dir)
    
    # Run GO
    useGroup <- "log2FoldChange"
    outName <- file.i.simp.name

    success <- "yes"
    gsea <- tryCatch({
        GO_run(file.i, useGroup, outName, 1, FALSE)
        message(paste(file.i, "failed",sep='; '))
        return("no")
    })
}


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] "KO_48h-KO_Naive-Salmon_gn    Gene number: 6076"


'select()' returned many:many mapping between keys and columns


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] "KO_48h-WT_48h-Salmon_gn    Gene number: 4448"


'select()' returned many:many mapping between keys and columns


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


...

'select()' returned many:many mapping between keys and columns


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] "WT_TE-WT_MP-Salmon_gn    Gene number: 2052"


'select()' returned 1:many mapping between keys and columns


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] "WT_TE-WT_Naive-Salmon_gn    Gene number: 4906"


'select()' returned many:many mapping between keys and columns

